In [87]:
import os,sys
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
import numpy as np
import FinanceDataReader as fdr
from pykrx import stock, bond
from datetime import datetime, timedelta

def get_date(types,day):
    if types == 'fdr':
        return (datetime.now()-timedelta(day)).strftime('%Y-%m-%d')
        
    else : 
        return (datetime.now()-timedelta(day)).strftime('%Y%m%d')
    
krx_raw = fdr.StockListing('KRX')
krx_raw.columns = [x.lower() for x in krx_raw.columns]
krx = krx_raw[krx_raw.sector.notnull()]

In [88]:
datefrom = get_date('pk',365)
now = get_date('pk',1)
fundam = stock.get_market_fundamental(now, market='ALL')
cap = stock.get_market_cap(now)
fun_cap = pd.merge(cap, fundam, on='티커')
info = pd.merge(krx,fun_cap, left_on='symbol',right_on='티커')
col_to = {'종가':'close', '시가총액':'cap','거래량':'volume',
           '거래대금':'amount','상장주식수' : 'shares_num'}
info = info.rename(columns=col_to)

In [ ]:
import time
m_fundm = pd.DataFrame()
for code in info.symbol:
    df = stock.get_market_fundamental(datefrom,now,code, freq="m")
    df['symbol'] = code 
    m_fundm = pd.concat([m_fundm,df])
    time.sleep(0.1)

In [97]:
info.to_csv('./stock_info.csv')
m_fundm.to_csv('./monthly_fundamental.csv')

In [ ]:
sector = dict(list(df.groupby('sector')))
sec_name = list(sorted(krx.sector.unique()))
sec_ = sector[sec_name[0]]
sec_df = pd.DataFrame()
for ix, row in sec_.iterrows() :
    code, name = row['symbol'], row['name']
    sec_value = fdr.DataReader(code,datefrom,now)
    sec_df[name] = sec_value['Close']

sec_df = sec_df.dropna(axis=1)

In [ ]:
row_dict ={}
for days in [5,10,20,60,120,240]:
    start = datetime.now() - timedelta(days)
    sec_day = sec_df[start:datetime.now()]
    stock_yield = (sec_day / sec_day.iloc[0] -1.0)*100
    row_dict[days] = stock_yield.iloc[-1]

df_yield = pd.DataFrame(row_dict)
print(sec_name[0])
df_yield